## Подготовка данных


Выбранный текст - Человек, который принял жену за шляпу

In [1]:
#constants 
file_name = 'Человек,который_принял_жену_за_шляпу.pdf' # реальный текст романа с 14 по 112 страницы 
sentence_len = 50

In [76]:
! pip install pdfquery
! pip install pandas
! pip install --user -U nltk

import sys
!{sys.executable} -m pip install numpy

! pip install torch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new 

In [4]:
#imports 
from pdfquery import PDFQuery

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import nltk

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string
import re

from torch.utils.data import Dataset, DataLoader, TensorDataset

In [78]:
pdf = PDFQuery(file_name)
pdf.load(*range(13, 30)) #TODO("увеличить?")

text_elements = pdf.pq('LTTextLineHorizontal')

In [79]:
text = "".join([t.text for t in text_elements]).replace('\n', '')

In [80]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/esoboleva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def tokenize_ru(file_text):
    # firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)

    # let's delete punctuation symbols
    tokens = [i for i in tokens if (i not in string.punctuation)]

    # deleting other symbols
    punct = ['—', ',', '.', '...']
    tokens = [i for i in tokens if (i not in punct)]

    # cleaning words
    tokens = [re.sub("[a-z]","", re.sub("[0-9]","",
              i.replace("«", "")
              .replace("»", "")
              .replace(".", "")
              .replace("-", "")
              .replace('—', "")
              .replace(',', "")
              .replace('`', "")
                                        
              .lower()))
              for i in tokens]
    

    return tokens

In [3]:
sentences = [tokenize_ru(sent) for sent in sent_tokenize(text, 'russian')]

NameError: name 'sent_tokenize' is not defined

In [100]:
alphabet = {'\n'}

loaded_text_path = 'text.txt'
text_file = open(loaded_text_path, "w")

for s in sentences:
    joined_s = " ".join(s)
    if len(joined_s) > sentence_len:
        text_file.write(joined_s)
        text_file.write('\n')
        # getting alphabet
        for c in joined_s:
            alphabet.add(c)
text_file.close()
alphabet = sorted(alphabet)
print(alphabet)

['\n', ' ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']


остались только кириллица и пробел - ура

сделаем one-hot encoding

In [102]:
text = open("text.txt").read()

In [103]:
char_to_int = dict((c, i) for i, c in enumerate(alphabet))

In [104]:
n_chars = len(text)
n_vocab = len(alphabet)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  71224
Total Vocab:  34


In [105]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = text[i:i + seq_length]
	seq_out = text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  71124


In [106]:
# reshape X to be [samples, time steps, features] 
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)
print(X.shape, y.shape)

torch.Size([71124, 100, 1]) torch.Size([71124])


## LSTM

In [109]:
class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(256, n_vocab)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(x)
        return x

In [110]:
n_epochs = 30
batch_size = 128
model = RNNModel()
 
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
loader = DataLoader(TensorDataset(X, y), shuffle=True, batch_size=batch_size)

best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss += loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss / batch_size))

Epoch 0: Cross-entropy: 13.0505
Epoch 1: Cross-entropy: 12.8368
Epoch 2: Cross-entropy: 12.7028
Epoch 3: Cross-entropy: 12.5713
Epoch 4: Cross-entropy: 12.4261
Epoch 5: Cross-entropy: 12.2570
Epoch 6: Cross-entropy: 12.0824
Epoch 7: Cross-entropy: 11.8200
Epoch 8: Cross-entropy: 11.6235
Epoch 9: Cross-entropy: 11.3298
Epoch 10: Cross-entropy: 11.0551
Epoch 11: Cross-entropy: 10.8084
Epoch 12: Cross-entropy: 10.5372
Epoch 13: Cross-entropy: 10.2898
Epoch 14: Cross-entropy: 10.0433
Epoch 15: Cross-entropy: 9.7807
Epoch 16: Cross-entropy: 9.5923
Epoch 17: Cross-entropy: 9.3388
Epoch 18: Cross-entropy: 9.0090
Epoch 19: Cross-entropy: 8.8162
Epoch 20: Cross-entropy: 8.5884
Epoch 21: Cross-entropy: 8.3983
Epoch 22: Cross-entropy: 8.2170
Epoch 23: Cross-entropy: 8.0129
Epoch 24: Cross-entropy: 7.7941
Epoch 25: Cross-entropy: 7.5952
Epoch 26: Cross-entropy: 7.4704
Epoch 27: Cross-entropy: 7.4840
Epoch 28: Cross-entropy: 7.1478
Epoch 29: Cross-entropy: 7.0355
900.5380249023438


In [112]:
torch.save([best_model, char_to_int], "model.pth")

In [3]:
file = open('text.txt', 'r')
lines = file.readlines()

In [114]:
prompt_len = 15
sentence = np.random.randint(len(lines))
prompt = lines[sentence][0:prompt_len]

In [115]:
pattern = [char_to_int[c] for c in prompt]

int_to_char = dict((i, c) for c, i in char_to_int.items())

model.eval()
print("Real sentence:\n", lines[sentence])
print('Prompt: "%s"' % prompt)
print(prompt, end="")
with torch.no_grad():
    for i in range(500):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x)
        # convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

Real sentence:
 я попросил профессора вообразить что он подходит к одной из наших площадей с севера

Prompt: "я попросил проф"
я попросил профессора п ракиесил м порертотетюни кан п м с но с н мрриц полазало т мого соеа не полушариим с лимо раз иетоос короя и стоанн поомес ле оонаже о болаз веестиве но все еа ррланаз те оо р селеном  пз зоие о брлр исийиоу ттади н сооеитине нореро порано пе днзе поерат челныеооие м п помогний и оенафоним моииа в оорееений ни пасвкини и коинил бно зап что ое тан втль и кр поимени о ракоти сторенно ие пиисториео иаиас помо посгреа отириятс  с пам стляко поиибног  оне оизедвлия иоторию болезни оакять аес
Done.


## Марковская цепь


In [4]:
n_chars_to_consider = 6
transitions = dict()

In [5]:
for s in lines:
    prev = None
    i = 0
    while i < len(s):
        if prev == None:
            prev = s[:n_chars_to_consider]
            i += n_chars_to_consider
            continue
        new_state = prev[1:] + s[i]
        key = prev + '->' + new_state
        if not (key in transitions):
            transitions[key] = 1
        else:
            transitions[key] += 1
        prev = new_state
        i+=1

# нормируем
summ = 0
for t in transitions.values():
    summ += t

transitions = {k: v / summ for k, v in transitions.items()}
print("Transitions count: ", len(transitions.items()))

Transitions count:  30779


In [6]:
prompt_len = 11 # > n_chars_to_consider

sentence = np.random.randint(len(lines))
prompt = lines[sentence][0:prompt_len]

NameError: name 'np' is not defined

In [119]:
# предсказание
import operator

def our_transition(pair, suf):
    k, v = pair 
    return k.startswith(suf)
        

def predict(cur_line):
    suf = cur_line[len(cur_line) - n_chars_to_consider:] 
    trs = dict()
    for t in transitions.items():
        k, v = t
        if our_transition(t, suf):
            trs[k] = v
    if len(trs) == 0:
        return None, True
    
    max_val = max(trs.values())
    max_prob_key = max(trs, key = trs.get)
    
    next_char = max_prob_key[len(max_prob_key) - 1] #last 
    return cur_line + next_char, False

print("Real sentence:\n", lines[sentence])
print('Prompt: "%s"' % prompt)
cur = prompt
for i in range(100):
    cur, ended = predict(cur)
    if ended:
        print('Достигли перехода в конец строки, всё')
        break
    print(cur)


Real sentence:
 он должен был мысленно пересечь ее и рассказать мне мимо каких зданий проходит

Prompt: "он должен б"
он должен бы
он должен был
он должен был 
он должен был м
он должен был мы
он должен был мыс
он должен был мысл
он должен был мысле
он должен был мыслен
он должен был мысленн
он должен был мысленно
он должен был мысленно 
он должен был мысленно п
он должен был мысленно пе
он должен был мысленно пер
он должен был мысленно пере
он должен был мысленно перес
он должен был мысленно пересе
он должен был мысленно пересеч
он должен был мысленно пересечь
он должен был мысленно пересечь 
он должен был мысленно пересечь е
он должен был мысленно пересечь ее
он должен был мысленно пересечь ее 
он должен был мысленно пересечь ее и
он должен был мысленно пересечь ее и 
он должен был мысленно пересечь ее и р
он должен был мысленно пересечь ее и ра
он должен был мысленно пересечь ее и рас
он должен был мысленно пересечь ее и расс
он должен был мысленно пересечь ее и расск
он должен был 